In [4]:
!git clone https://github.com/taslimamindia/Tp2_IFT870.git

fatal: destination path 'Tp2_IFT870' already exists and is not an empty directory.


In [5]:
#importations de bibliothèques
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

ValueError: Name tf.RaggedTensorSpec has already been registered for class tensorflow.python.ops.ragged.ragged_tensor.RaggedTensorSpec.

In [ ]:
# chargement des deux tables de la base de données
orders_distance_stores_softmax = pd.read_csv("Tp2_IFT870/orders_distance_stores_softmax.csv")
orders_products__prior_specials = pd.read_csv("Tp2_IFT870/order_products__prior_specials.csv")

In [ ]:
orders_distance_stores_softmax

In [ ]:
orders_products__prior_specials

## Concaténation des deux tables

In [ ]:
# Obtenir l'ensemble des noms de colonnes uniques
colonnes_uniques = set(orders_distance_stores_softmax.columns).union(set(orders_products__prior_specials.columns))

Pour la concaténation, nous avons utilisé la méthode "merge" en se basant sur l'attribut commun des deux tables "order_id". Nous avons fixé la méthode de jointure à "inner" pour obtenir uniquement les lignes qui ont des correspondances dans les deux tables.

In [ ]:
#combiner les deux tables de la base de données
data = orders_products__prior_specials.merge(orders_distance_stores_softmax, on='order_id', how='inner', suffixes=('', '_dup'))


In [ ]:
data.shape

In [ ]:
data.head()

Notre ensemble de données contient 1172312 et 15 colonnes dont les colonnes "Unnamed". Nous allons les supprimer (non pertinents pour notre étude).

In [ ]:
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 0_dup'])

In [ ]:
data.columns

Nous avons donc 13 attributs donc l'attribut à prédire "reordered". Nous allons afficher le nombre d'échantillons pour chaque classe de l'attribut. Cette information sera pertinente pour l'analyse des résultats de prédiction. En cas de déséquilibre, il faut choisir des métriques spécifiques comme F1.

In [ ]:
data['reordered'].value_counts()

## Réduction des données (5%)

Avant de réduire nos données, nous allons les pourcentages des valeurs manquantes pour optimiser notre nouvel sous-ensemble.

In [ ]:
# Calcul de la moyenne des valeurs manquantes pour chaque colonne
pourcentage_valeurs_manquantes = (data.isnull().mean() * 100).round(2)

# Convertir le résultat en DataFrame pour une meilleure présentation
tableau_pourcentage_valeurs_manquantes = pd.DataFrame(pourcentage_valeurs_manquantes, columns=['Pourcentage de valeurs manquantes'])

tableau_pourcentage_valeurs_manquantes

Puisque nous avons plus de 1m de lignes (échantillons), nous nous permettons de supprimer les lignes où la valeur 'days_since_prior_order' est manquante.

In [ ]:
# Suppression des lignes où 'days_since_prior_order' est null
data = data.dropna(subset=['days_since_prior_order'])

Pour générer un sous ensemble de 5% des données, nous avons utilisé la méthode "train_test_split". Nous avons fait une stratification par rapport au "user_id" parce que notre prédiction de l'attribut "reordered" concerne aussi l'utilisateur (s'il a refait la même commande) donc nous avons voulu assurer qu'il y a un nombre équilibré et varié des utilisateurs (et leurs commandes).

In [ ]:
# stratification avec les IDs des utilisateurs
_, sampled_df = train_test_split(data, test_size=0.05, stratify=data['user_id'], random_state=42)

# Sous ensemble obtenu
print(sampled_df.columns)

In [ ]:
sampled_df.shape

Notre sous-ensemble contient 57923 échantillons et 12 prédicteurs.

In [ ]:
#afficher quelques statistiques sur les attributs du sous ensemble obtenu
sampled_stats = sampled_df.describe()
print(sampled_stats)

Nous allons vérifier qu'il y a un nombre raisonnable des commandes par utilisateur.

In [ ]:
# Calcul de la taille de chaque groupe d'user_id
group_sizes = sampled_df.groupby('user_id').size()

# Tentative d'afficher tous les résultats
print(group_sizes.to_string())

## Préparation des données

Encodage des variables catégorielles (eval_set)

In [ ]:
# Initialisation du LabelEncoder
label_encoder = LabelEncoder()

# Identification des colonnes catégorielles
variables_categorielles = sampled_df.select_dtypes(include=['object', 'category']).columns

# Boucle sur chaque colonne catégorielle pour appliquer le LabelEncoder directement sur la colonne originale
for col in variables_categorielles:
    # Application du LabelEncoder à la colonne et écrasement de la colonne originale avec la version encodée
    sampled_df[col] = label_encoder.fit_transform(sampled_df[col])

In [ ]:
sampled_df.head

In [ ]:
#catégories pour les attributs de notre ensemble de données
sampled_df.nunique()

Nous remarquons que "eval_set" a une seule valeur.

In [ ]:
#vérifier que le LabelEncoder a été fonctionnel
sampled_df['eval_set'].dtype

Nous allons convertir les valeurs de l'attribut 'order_hour_of_day' pour qu'elles aient la même unité que l'attribut 'days_since_prior_order'. Nous voulons que les deux attributs soient exprimés en jours.

In [ ]:
# Convertir les heures en jours (1 jour = 24 heures) pour 'order_hour_of_day'
sampled_df['order_hour_of_day'] = sampled_df['order_hour_of_day'] / 24

## Vérification des valeurs dupliquées

In [ ]:
#vérifier qu'il n'y a pas de lignes (échantillons) dubliquées pour notre ensemble de données
duplicates_df = sampled_df[sampled_df.duplicated()]
duplicates_df_sorted = duplicates_df.sort_values(by=['order_id'])
print(f"Nombre de valeurs dupliquées dans nos données \n{duplicates_df_sorted.sum()}")

Dans cette partie, nous avons vérifié que nos échantillons ne soient pas dupliqués puisque tous les nombres sont à 0.

## Visualisation des données

In [ ]:
# Sélection des colonnes numériques
colonnes_numeriques = sampled_df.select_dtypes(include=['float64', 'int64', 'int32']).columns

# Définition de la taille de la grille
n_cols = 3
n_rows = int(np.ceil(len(colonnes_numeriques) / n_cols))

# Création des figures de boxplot avec ajustement de la taille
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 6))  # Ajuster la largeur et la hauteur selon vos préférences
axes = axes.flatten()

# Traçage des boxplots pour les colonnes numériques
for i, col in enumerate(colonnes_numeriques):
    sns.boxplot(y=sampled_df[col], ax=axes[i])
    axes[i].set_title(f'Boxplot de {col}')
    axes[i].set_ylabel(col)

# Masquer les axes supplémentaires s'il y en a
for j in range(i + 1, n_rows * n_cols):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

Analyse des visualisations des boxplots :

Boxplot de store_id : Les identifiants de magasin montrent une certaine variabilité, mais la distribution est centrée, ce qui indique que les commandes proviennent d'un groupe relativement cohérent de magasins.

Boxplot de order_number : Les numéros de commande semblent avoir une distribution étendue avec la médiane vers le bas de la gamme, ce qui pourrait indiquer que de nombreux utilisateurs sont dans les premières étapes de l'historique de commande.

Boxplot de days_since_prior_order : Cette variable a une médiane proche de 0 avec des valeurs étendues vers le haut, ce qui indique des variations dans le temps écoulé depuis la dernière commande.

Boxplot de product_id : Les identifiants de produit sont distribués sur une large gamme avec quelques valeurs aberrantes, ce qui indique une grande variété de produits commandés.

Boxplot de special : Cette variable a une médiane à 0 et quelques valeurs aberrantes, ce qui suggère que des offres spéciales ne sont appliquées que dans un nombre limité de cas.

Boxplot de distance : Il y a des valeurs aberrantes, ce qui indique que bien que la plupart des utilisateurs soient à une distance moyenne des magasins, certains sont nettement plus éloignés.

Boxplot de order_dow : La distribution semble régulière sur toute la semaine sans valeurs aberrantes, suggérant une tendance uniforme dans les jours de passation des commandes.

Boxplot de add_to_cart_order : Il y a une grande variété dans l'ordre d'ajout des produits au panier, avec de nombreuses valeurs aberrantes, ce qui pourrait indiquer que certains produits ont tendance à être ajoutés au panier avant d'autres.

Boxplot de user_id : Les identifiants d'utilisateur ont une large distribution, ce qui suggère une grande base d'utilisateurs.

Boxplot de eval_set : Le boxplot n'est pas informatif ici puisque l'attribut contient une seule valeur.

Boxplot de order_hour_of_day : La distribution des heures de commande est centrée avec quelques valeurs aberrantes, indiquant des moments privilégiés pour passer commande au cours de la journée.

## Feature Engineering

Dans ce contexte de ce tp, vous souhaitez prédire si un produit sera réordonné (reordered étant la variable cible) dans une future commande en fonction de l'historique d'achat d'un utilisateur. Basé sur cette objectif, voici les attributs que nous pensons ne pas avoir de valeur ajoutée pour la prédiction et donc être des candidats à l'élimination :

- order_id : C'est un identifiant unique pour chaque commande, il n'aide pas directement à prédire le réordonnancement.

- eval_set : Il s'agit simplement d'une variable technique indiquant l'appartenance des données à un ensemble d'entraînement, de validation ou de test et qui garde la même valeur pour toutes les échantillons de notre ensemble de données. Elle ne doit pas être incluse dans l'entraînement du modèle pour éviter la fuite de données.

- store_id : L'identifiant du magasin n'a pas de relation avec la probabilité de réordonner un produit, cet attribut est à écarter.

L'attribut "user_id" en soi n'est pas une caractéristique prédictive directe, mais il est essentiel pour grouper les données et générer des caractéristiques comportementales significatives au niveau de l'utilisateur qui peuvent grandement améliorer la performance de votre modèle de prédiction.

In [ ]:
# Suppression des attributs en question
sampled_df = sampled_df.drop(columns = ['store_id', 'order_id', 'eval_set'])

## Normalisation des données

Cette étape est importante et nécessaire pour l'entraînement de la majorité des modèles de Machine et/ou Deep Learning.

In [ ]:
# Préparation des ensembles X et y
X = sampled_df.drop('reordered', axis=1)
y = sampled_df['reordered']

# Création de l'objet StandardScaler pour la normalisation
scaler = StandardScaler()

# Ajustement du scaler sur les données puis transformation
X_scaled = scaler.fit_transform(X)

# Convertir X_scaled de nouveau en DataFrame pour garder les noms de colonne
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

## Division des données en ensembles Train et Test

In [ ]:
# Séparation des données en ensemble d'entraînement (80%) et de test (20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Sélection des colonnes numériques pour des visualisations
numeric_columns = X_train.select_dtypes(include=['float64', 'int64']).columns

In [ ]:
# Réglage des paramètres pour afficher 3 figures par ligne
n_cols = 3  # Nombre de colonnes par ligne dans la grille de subplots
n_rows = int(np.ceil(len(numeric_columns) / n_cols))  # Calcul du nombre de lignes nécessaires

# Création d'une figure et de subplots avec la grille spécifiée
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 5))

# Aplatir le tableau d'axes pour une itération facile si nécessaire
axes_flat = axes.flatten()

# Visualisation avec des histogrammes pour chaque colonne numérique
for i, col in enumerate(numeric_columns):
    sns.histplot(X_train[col], color='blue', label='Train', kde=True, ax=axes_flat[i])
    sns.histplot(X_test[col], color='orange', label='Test', kde=True, ax=axes_flat[i])
    axes_flat[i].set_title(f'Distribution de {col} pour train et test')
    axes_flat[i].legend()

# Si le nombre total de colonnes numériques n'est pas un multiple de 3, masquer les axes vides
for j in range(i+1, len(axes_flat)):
    axes_flat[j].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Création d'une figure avec la grille de subplots pour des courbes de densité
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 6, n_rows * 4))

# Parcourir chaque colonne numérique et tracer des courbes de densité pour les ensembles train et test
for i, col in enumerate(numeric_columns):
    row = i // n_cols
    col_index = i % n_cols
    ax = axes[row, col_index]

    # Tracer la courbe de densité pour l'ensemble d'entraînement
    sns.kdeplot(data=X_train, x=col, fill=True, ax=ax, label='Train', color='blue', alpha=0.5)

    # Tracer la courbe de densité pour l'ensemble de test
    sns.kdeplot(data=X_test, x=col, fill=True, ax=ax, label='Test', color='orange', alpha=0.5)

    ax.set_title(f'Distribution de {col}')
    ax.legend()

# Cacher les subplots vides s'il y en a
for j in range(i+1, n_rows*n_cols):
    axes.flat[j].set_visible(False)

plt.tight_layout()
plt.show()

Les courbes montrent que les ensembles d'entraînement et de test ont des distributions cohérentes et comparables pour la majorité des attributs examinées. Cela signifie que la séparation des données a été bien réalisée, offrant ainsi une bonne représentativité et permettant d'anticiper que les modèles de machine learning entraînés sur l'ensemble d'entraînement devraient avoir des performances similaires sur l'ensemble de test. Les pics et creux correspondants dans les courbes de densité de product_id, add_to_cart_order, et days_since_prior_order reflètent des tendances spécifiques de comportement d'achat qui sont préservées entre les ensembles. Les caractéristiques périodiques comme order_dow et les pics d'heures dans order_hour_of_day sont particulièrement remarquables, révélant des schémas d'achat récurrents sur les jours et les heures qui pourraient être significatifs pour la prédiction du réordonnancement. En résumé, ces visualisations fournissent une confirmation visuelle que la séparation des données maintient l'intégrité statistique nécessaire pour une modélisation prédictive fiable.

# Sauvegarde des fichiers

In [ ]:
# Pour les données d'entraînement
X_train.to_csv('x_train_final.csv', index=False)
y_train.to_csv('y_train_final.csv', index=False)

# Pour les données de test
X_test.to_csv('x_test_final.csv', index=False)
y_test.to_csv('y_test_final.csv', index=False)

# Pour sample_df
sampled_df.to_csv('final_data_1.csv', index=False)

## Entraînement des modèles (Question 2)

Avant d'entraîner nos données sur des modèles, nous allons voir la distribution des classes de l'attribut à prédire (reordered).

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

Nous avons choisi de ne pas appliquer de méthodes spécifiques pour équilibrer les classes de l'attribut à prédire. Nous traitons un problème de prédiction de re-commande d'un produit donc il est important que nous gardons les tendances telles qu'elles sont sans modification externe.


Puisque la couche de sortie utilise deux neurones, nous devons lui fournir un vecteur binaire à deux dimensions, c'est-à-dire contenant soit 0 ou 1. Cependant, notre vecteur cible est unidimensionnel. Par conséquent, nous allons utiliser un encodeur OneHotEncoder pour le transformer en un vecteur avec les dimensions appropriées.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
y_train_t, y_test_t = y_train, y_test
y_train_ = np.reshape(y_train, (-1, 1))
y_test_ = np.reshape(y_test, (-1, 1))

enc = OneHotEncoder(sparse_output=False)
y_train = enc.fit_transform(y_train_)
y_test = enc.fit_transform(y_test_)
print(y_train.shape, y_test.shape)

Implémentation du CNN

In [ ]:
# chaque exemple de notre ensemble de données a N caractéristiques
# Remodeler les données pour le CNN 1D
n_features = X_train.shape[1]  # N sera le nombre total de caractéristiques
X_train_reshaped = X_train.values.reshape(-1, n_features, 1)
X_test_reshaped = X_test.values.reshape(-1, n_features, 1)
print(X_train_reshaped.shape, X_test_reshaped.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Initialiser le modèle séquentiel
model = Sequential()

# Ajouter la première couche convolutive 1D
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_features, 1)))

# Ajouter une deuxième couche convolutive 1D
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))

# Ajouter une autre couche de MaxPooling
model.add(MaxPooling1D(pool_size=2))

# Aplatir les sorties pour les rendre compatibles avec les couches denses
model.add(Flatten())

# Ajouter une couche dense cachée
model.add(Dense(100, activation='relu'))

# Ajouter la couche de sortie
model.add(Dense(2, activation='softmax'))  # Utiliser 'sigmoid' pour la classification binaire

# Afficher le résumé du modèle
model.summary()

Nous utilisons le score f1 car nous avons des classes non équilibrées et dans ce cas, la précision (accuracy) ne permet pas de capturer la vraie performance du modèle. Et,  car il assure que les modèles doivent bien performer à la fois sur les classes majoritaires et minoritaires pour obtenir un score élevé.
En effet, dans les jeux de données déséquilibrés, améliorer la précision peut souvent se faire au détriment du rappel, et vice versa. Par exemple, un modèle pourrait prédire très conservativement les cas positifs (pour être sûr à 100% de leur positivité), augmentant ainsi la précision mais réduisant le rappel. Le F1-score, en tant que moyenne harmonique, ne peut être élevé que si les deux, précision et rappel, sont élevés, forçant ainsi le modèle à maintenir un équilibre entre eux.

In [ ]:
def f1_score(y_true, y_pred):
    y_pred = tf.math.round(y_pred)  # Arrondir les prédictions à 0 ou 1
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float'), axis=0)
    tn = tf.reduce_sum(tf.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + tf.keras.backend.epsilon())
    r = tp / (tp + fn + tf.keras.backend.epsilon())

    f1 = 2 * p * r / (p + r + tf.keras.backend.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return tf.reduce_mean(f1)

model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Utiliser 'binary_crossentropy' pour la classification binaire
              metrics=['accuracy', f1_score])

In [ ]:
# Pour la classification binaire, assurez-vous que y_train et y_test sont binaires (0 ou 1)
# Entrainement du modèle CNN
history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=64, validation_split=0.2, validation_data=(X_test, y_test))

Le modèle a pour f1-score 87.9%. Nous allons faire des tests manuels pour optimiser la performance du modèle CNN. Voir fichier **tp2-cnn_tests.ipynb** pour plus de détails. Nous avons utilisé un différent fichier pour faire les tests sur CNN parce que ça nécessitait trop de mémoire et c'est plus optimal et intéressant de faire ces calculs sur Colab. Nous avons obtenu le meilleur résultat pour ces valeurs de paramètres : "Filtres: 64, Taille de noyau: 2, Taux de dropout: 0.5, Optimiseur: rmsprop, LR: 0.001 " pour un F1-Score: 0.8. Nous allons tous de même présenter ces résultats en détails dans la partie suivante.

Le nombre de Filtres est égal à 64. Le nombre de filtres dans une couche convolutive détermine combien de caractéristiques uniques le modèle peut extraire à chaque couche. Un total de 64 filtres permet au modèle de capturer une grande variété de caractéristiques sans être trop complexe, ce qui pourrait mener à un surapprentissage, ni trop simple, ce qui limiterait sa capacité à apprendre des caractéristiques fines des données.

La taille de noyau est égale à 2. La taille du noyau affecte la quantité d'information que le filtre considère à chaque étape de convolution. Une taille de noyau de 2 permet au modèle de détecter des motifs très locaux, ce qui est particulièrement utile pour capter les relations à court terme dans les données, comme les commandes qui se suivent d'un seul client ou les tendances de commandes des clients sur une période précise.

Le taux de dropout est égal à 0.5. Le dropout est une technique de régularisation utilisée pour éviter le surapprentissage en "éteignant" aléatoirement une proportion de neurones durant l'entraînement. Un taux de 0.5 signifie que la moitié des activations sont annulées à chaque mise à jour. Cela force le réseau à apprendre des caractéristiques robustes en utilisant seulement une fraction des neurones, améliorant ainsi la généralisation.

L'optimiseur optimal est RMSprop. Il ajuste le taux d'apprentissage de manière adaptative pour chaque paramètre. Il est particulièrement efficace pour naviguer dans des paysages de gradients complexes et pour des problèmes où le gradient peut changer de manière abrupte, ce qui le rend bien adapté à de nombreux types de données de CNN.

LR (Learning Rate) est égal à 0.001. Le taux d'apprentissage détermine la taille des pas que l'optimiseur prend dans le paysage des gradients. Un taux de 0.001 est suffisamment petit pour permettre des ajustements précis dans les poids du modèle sans sauter par-dessus des minima locaux, mais assez grand pour assurer une convergence rapide. Ce taux est souvent choisi comme point de départ pour de nombreux types de réseaux neuronaux.

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

# Construction du modèle CNN avec les paramètres spécifiés
model_cnn = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model_cnn.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entraînement du modèle CNN avec early stopping
history = model_cnn.fit(X_train_reshaped, y_train, epochs=100, batch_size=64, verbose=0,
                        validation_split=0.2, callbacks=[early_stopping])

# Prédiction sur l'ensemble de test
predictions_cnn = model_cnn.predict(X_test_reshaped)
predictions_cnn = (predictions_cnn > 0.5).astype(int)

In [ ]:
# Calcul des métriques
accuracy = accuracy_score(y_test, predictions_cnn)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, predictions_cnn, average='macro')
# y_test_single_label = np.argmax(y_test, axis=1)
# predictions_cnn_single_label = np.argmax(predictions_cnn, axis=1)
# conf_matrix = confusion_matrix(y_test_single_label, predictions_cnn_single_label)

from sklearn.metrics import multilabel_confusion_matrix

confusion_matrix = multilabel_confusion_matrix(y_test, predictions_cnn, labels=[0, 1])
# Affichage des métriques
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Affichage de la matrice de confusion
print("Matrice de confusion :")
print(confusion_matrix)

# Affichage des courbes d'apprentissage
plt.figure(figsize=(14, 5))

# Courbe d'exactitude
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Courbe d\'exactitude')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Courbe de perte
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Courbe de perte')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
!pip install lazypredict

## Entraînement de modèles Machine Learning

Résultat préliminaire avec la bibliothèque LazyPredict.

In [ ]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

# Initialisation de LazyClassifier pour comparer les modèles de classification
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train_t, y_test_t)

# Affichage des scores F1 des modèles
print(models['F1 Score'])

Nous remarquons que les F1-scores sont assez proches. Le LGBM (Light Gradient Boosting Model) est le modèle le plus performant avec 0.79 de f1-score. Il est suivi par plusieurs modèles ayant comme f1-score 0.78. Nous allons en choisir quelques uns pour les optimiser avec GridSearchCV. Pour cela, nous allons travailler sur les modèles LGBMClassifier, AdaBoostClassifier, LinearDiscriminantAnalysis et RandomForestClassifier. Lazypredict permet en effet d'avoir des résultats préliminaires sur une multitude de modèles de Machine Learning (utilisant plusieurs techniques comme le Bagging, Boosting et autres techniques d'apprentissage).
Nous allons tout de même commencer par optimiser le modèle SVM puisque c'est demandé dans l'énoncé.

## SVM

Nous implémentons la méthode de classification Machine à vecteur de support, avec un gridsearch pour la recherche des meilleurs hyper-parametres (Le noyeau, gamma et la constante C).

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category = ConvergenceWarning)

#Différentes valeurs pour différents hyperparamètres du modèle SVM
param_grid = {
    'C': [0.1, 1, 10, 100],  # Plus C est élevé, moins la régularisation est forte
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly']  # différents noyaux
}

svc = SVC(max_iter=200)  # Initialiser le modèle SVM

grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, refit=True, verbose=2, cv=5)

L'entrainement de la grille de recherche pour trouver le meilleur modèle avec les bons hyper-parametres.

In [ ]:
grid_search.fit(X_train, y_train_t)

In [ ]:
print(f"Meilleurs paramètres: {grid_search.best_params_}")
best_model = grid_search.best_estimator_

# Prédiction et rapport de classification sur l'ensemble de test
predictions = best_model.predict(X_test)
print(classification_report(y_test_t, predictions))